In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
from keras.utils import to_categorical

# Define the directory where the folders of images are located
directory = '/content/drive/My Drive/Dataset_Raw/Pneumonia_vs_Normal/'

# Define the desired size for the images
image_size = (28, 28)

# Define the mapping of folder names to numerical labels
class_dict = {'Normal': 0, 'Pneumonia': 1}

# Define the number of classes
num_classes = len(class_dict)

# Load the images and preprocess them
data = []
labels = []
for class_name in os.listdir(directory):
    class_directory = os.path.join(directory, class_name)
    if class_name in class_dict:
        class_label = class_dict[class_name]
        for filename in os.listdir(class_directory):
            if filename.endswith('.png'):
                img = cv2.imread(os.path.join(class_directory, filename))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # convert to grayscale
                img = cv2.resize(img, image_size)  # resize the image
                data.append(img)
                labels.append(class_label)

# Convert the data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Reshape the data to have a single channel (grayscale)
data = np.reshape(data, (*data.shape, 1))

# Convert the labels to one-hot encoding
labels = to_categorical(labels, num_classes)



In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.naive_bayes import GaussianNB

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model on the training set
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Extract features from the CNN model for the training and testing sets
train_features = model.predict(x_train)
test_features = model.predict(x_test)

nb = GaussianNB()
nb.fit(train_features, np.argmax(y_train, axis=1))

Epoch 1/10
68/68 [==============================] - 4s 41ms/step - loss: 1.0527 - accuracy: 0.8778 - val_loss: 0.1071 - val_accuracy: 0.9572
Epoch 2/10
68/68 [==============================] - 2s 37ms/step - loss: 0.0605 - accuracy: 0.9809 - val_loss: 0.0477 - val_accuracy: 0.9851
Epoch 3/10
68/68 [==============================] - 2s 36ms/step - loss: 0.0591 - accuracy: 0.9805 - val_loss: 0.0596 - val_accuracy: 0.9777
Epoch 4/10
68/68 [==============================] - 4s 55ms/step - loss: 0.0278 - accuracy: 0.9912 - val_loss: 0.0346 - val_accuracy: 0.9870
Epoch 5/10
68/68 [==============================] - 3s 49ms/step - loss: 0.0133 - accuracy: 0.9958 - val_loss: 0.0528 - val_accuracy: 0.9814
Epoch 6/10
68/68 [==============================] - 2s 36ms/step - loss: 0.0136 - accuracy: 0.9949 - val_loss: 0.0237 - val_accuracy: 0.9907
Epoch 7/10
68/68 [==============================] - 2s 36ms/step - loss: 0.0066 - accuracy: 0.9981 - val_loss: 0.0174 - val_accuracy: 0.9907
Epoch 8/10
68

GaussianNB()

In [5]:
accuracy = nb.score(test_features, np.argmax(y_test, axis=1))
print('Naive Bayes classifier accuracy:', accuracy)

Naive Bayes classifier accuracy: 0.9888475836431226
